In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as pt

## Query data

In [113]:
def query_graph(uri: str, query: str) -> dict:
    request = requests.post(uri, json={'query': query})
    if request.status_code >= 200 and request.status_code <= 299:
        return request.json()
    else:
        raise Exception(f"Failed HTTP Post request with status: {request.status_code}")

### Repuation Holders
First query all reputation holders and filter by organization to get dOrg members

In [178]:
def queryAllRepHolders() -> list:
    result = []
    url = 'https://api.thegraph.com/subgraphs/name/daostack/alchemy'
    skip = 0
    query = '{{ \n \
      reputationHolders(first: 1000, skip:{skip}) {{ \n \
        address \n \
        balance \n \
        dao {{ \n \
          name \n \
        }} \n \
        createdAt \n \
      }} \n \
    }}'
    response = query_graph(url, query.format(skip=skip))
    while len(response['data']['reputationHolders']) > 0:
        result += response['data']['reputationHolders']
        skip += 1000
        response = query_graph(url, query.format(skip=skip))
    return result

In [179]:
response_reputation_holders = queryAllRepHolders()

In [180]:
def filter_rep_holders_by_dao(df, dao_name):
    undict = lambda x: x['name']
    df['dao'] = df['dao'].apply(undict)
    return df[df.dao == dao_name]

In [181]:
reputation_holders = pd.DataFrame(response_reputation_holders)
reputation_holders = filter_rep_holders_by_dao(reputation_holders, 'dOrg')

In [182]:
reputation_holders

,address,balance,createdAt,dao
85,0x5db06acd673531218b10430ba6de9b69913ad545,80840000000000000000,1581467584,dOrg
210,0x3e9976d5ba86a78d6e5c25bc2f309049676c0798,24633000000000000000,1560273875,dOrg
217,0xc6ee47385c205139612a4f847a067bfcdc59df54,113114999999999998437,1573308936,dOrg
284,0x0a0b5fe27b2f4be657ee91c48af4ceeae097706e,48340000000000000746,1575301619,dOrg
450,0x1338c277e03fbe9d6b1b3b655f0e567c0dcaac4a,1428000000000000000000,1581539885,dOrg
...,...,...,...,...
6426,0x3e9976d5ba86a78d6e5c25bc2f309049676c0798,4926000000000000000000,1581539885,dOrg
6444,0x1f134294dee0055da2a9914f73d2a7a59d374ac0,2200000000000000000000,1581539885,dOrg
6547,0x5b52ba08252f17a1560b8f6bdf025518d18b6a62,4050000000000000000000,1583472751,dOrg
6563,0xbeeb66591fc66210672880da41a481bf9fcfc72a,18372000000000000000000,1581539885,dOrg


### Repuation Transactions

In [222]:
def queryBlockRepTransactions(table: str, block=10718611) -> list:
    result = []
    url = 'https://api.thegraph.com/subgraphs/name/daostack/alchemy'
    skip = 0
    query = '{{ \n \
        {table} (first: 1000, block: {{ number: {block} }}, skip: {skip}) {{ \n \
        id \n \
        txHash \n \
        contract \n \
        address \n \
        amount \n \
      }} \n \
    }}'
    response = query_graph(url, query.format(table=table, block=block, skip=skip))
    while len(response['data'][table]) > 0:
        result += response['data'][table]
        skip += 1000
        response = query_graph(url, query.format(table=table, block=block, skip=skip))
    return result

In [223]:
def queryAllRepTransactions(table: str, newest_block: int, address_whitelist=None):
    result = pd.DataFrame()
    block = newest_block
    response = queryBlockRepTransactions(table, block)
    while len(response) > 0 and block > 10718611-50:
        # add to result
        response_df = pd.DataFrame(response)
        if address_whitelist is not None:
            response_df = [response_df.address.isin(address_whitelist)]
        response_df['block'] = block
        result = result.append(response_df)
        result = result.drop_duplicates(subset=['id', 'txHash', 'contract', 'address', 'amount'], keep='first')
        # query next block
        block -= 1
        response = queryBlockRepTransactions(table, block)
    result.reset_index(inplace=True)
    return result

In [225]:
response_reputation_mints = queryBlockRepTransactions('reputationMints')

In [229]:
reputation_mints = pd.DataFrame(response_reputation_mints)
reputation_mints

,address,amount,contract,id,txHash
0,0x0ebbda43014a6e6c32657a8db79376738d549b2f,57686312682801532550,0x30412f4c06343d65c06b5a0bba6821cf37e605fa,0x0002a6074543687338a3de9f9a53e3382667334009cd...,0x48e571535ed433e45c0d8a49c2d5afa62f1832f95c6c...
1,0x3f0b2a18b6da197c8c0744918febc7dcddab0e04,30000000000000000000,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,0x0008dbd90effb817853281b7ce592fff48a35e7aaf89...,0xb705eeb2b9cd9de888662901dfaf1d50f083e5fbda73...
2,0x81aaa9a7a8358cc2971b9b8de72acce6d7862bc8,50000000000000000000,0x30412f4c06343d65c06b5a0bba6821cf37e605fa,0x000e69e1ef68b56fb5ce1870291b5dfc9882ffd73420...,0x7a1da0b3d9c7a6cbb4cc73545fd8281bd64f517cdb85...
3,0xb9e29984fe50602e7a619662ebed4f90d93824c7,130000000000000000000,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,0x001617af3e786cbc864c8fa938ed74cae8a4bc8ea7cb...,0x629a7a3b6c70552a7f1e711841aae2ab6627e5ebf73e...
4,0xb7bdb99abbddebba1d5802c0f4e5073f1a75a9ea,10000000000000000000,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,0x0022f022c026f08e5f70c2cc9362507cdceddbc9ddef...,0xdc26e17fe7fef5fdceacbe08f06b8f440efadbbc191c...
...,...,...,...,...,...
9998,0x24597c5c68687e816ffc0c69e064cb70bb62a9cd,33312912412375480946,0x30412f4c06343d65c06b5a0bba6821cf37e605fa,0xffce060627e5e8bc25132eabcc5c2d3ce9834b933fb4...,0x28a6601382fd0cfe83d11876159f5ebaaffa2a8a5e46...
9999,0x30043aabbcebbd887437ec4f0cfe6d4c0eb5cc64,5794387245460010576,0x30412f4c06343d65c06b5a0bba6821cf37e605fa,0xffd282038b45de10e3c64f1ca9d15c20204a441286aa...,0x8b026c10ea6f2cf9f3f562ea7a3ab707d24c17b46cb7...
10000,0x1206ec7abf2c3d22051f6ebb249f065fc6436149,50000000000000000000,0x30412f4c06343d65c06b5a0bba6821cf37e605fa,0xffd664c72f814a7f80cee8f3c9373870e8ccd4c8ee4f...,0x82d919691c45dc59817ea2a69600cc6e62ee340e55e8...
10001,0x19cec11d268d61305e53524a55f8e94733c98d96,50000000000000000000,0xed1c0bb038b370a16242f077899ee1e5e7d6add1,0xffeddfe5fa020576815656f804fc8ac0de3313b2a9ee...,0x0a54643070242ff667d37908a31cc5ee3cc3c68e2d71...
